# Implementation of "Fair Decisions Despite Imperfect Predictions" using the original fairness constraint

In [1]:
import numpy as np
from src.consequential_learning import collect_data, train
from src.feature_map import IdentityFeatureMap

## The parameters used by the the original authors  
Note: Learning rate decay is not yet implemented

In [7]:
training_parameters = {
    'dim_x': 1,
    'dim_s': 1,
    'time_steps':200,
    'batch_size':512,
    'num_iterations': 32,
    'learning_rate':0.5,
    'fairness_rate':0,
    'cost_factor':0.55,
    'fraction_protected':0.3
}
training_parameters['dim_theta'] = training_parameters['dim_x'] + training_parameters['dim_s']
training_parameters['feature_map'] = IdentityFeatureMap(training_parameters['dim_theta'])
training_parameters['num_decisions'] = training_parameters['num_iterations'] + training_parameters['batch_size']

## Definition of the fairness function  
The original fairness constraint was defined as the difference of benefits $b_{P}^s$ for both of the settings of the protected attribute. This function defines both the fairness function as well as its derivative which is controlled by the parameter gradient=true/false

In [8]:
def benefit_function(x_s, s, sample_theta, policy, gradient):
    ips_weight, phi, log_gradient_denominator = policy.calculate_ips_weights_and_log_gradient(x_s, s, sample_theta)
    decision = policy(x_s, s).reshape(-1, 1)

    if gradient:
        grad_benefit = ((ips_weight/log_gradient_denominator) * decision * phi).sum(axis=0) / x_s.shape[0]
        return grad_benefit
    else:
        benefit = (ips_weight * decision).sum(axis=0) / x_s.shape[0]
        return benefit

def fairness_function(**fairness_kwargs):
    x = fairness_kwargs['x']
    s = fairness_kwargs['s']
    sample_theta = fairness_kwargs['sample_theta']
    policy = fairness_kwargs['policy']
    gradient = fairness_kwargs['gradient']

    pos_decision_idx = np.arange(s.shape[0]).reshape(-1, 1)

    s_0_idx = pos_decision_idx[s == 0]
    s_1_idx = pos_decision_idx[s == 1]

    return benefit_function(x[s_0_idx], s[s_0_idx], sample_theta, policy, gradient) - benefit_function(x[s_1_idx], s[s_1_idx], sample_theta, policy, gradient)


## Train the model

In [4]:
train(**training_parameters, fairness_function=fairness_function)

NameError: name 'DIM_S' is not defined